In [3]:
import requests     
import json         
import pandas as pd 
import numpy as np
import time
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [23]:
path = "./Data/withFIGI.csv"

df = pd.read_csv(path, index_col=0)  
df

,Name_197,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,...,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI,Not_Match
0,1ST GROUP LTD,1ST,Australia,AUD,AU,AU0000001ST0,1stAvailable Ltd,Common Stock,0.010,2022-08-03,...,1ST,BBG008HNLRF4,BBG008HNLRF4,BBG008HNLRG3,AU,Equity,1ST,Common Stock,Common Stock,NaN
1,2U INC,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,...,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock,NaN
2,3DX INDUSTRIES INC,DDDX,USA,USD,US,US88556A1007,3Dx Industries Inc,Common Stock,0.060,2022-08-02,...,DDDX,BBG000C129N6,BBG000C129N6,BBG001STRH79,US,Equity,DDDX,Common Stock,Common Stock,NaN
3,3M CO,MMM,USA,USD,US,US88579Y1010,3M Company,Common Stock,141.750,2022-08-02,...,MMM,BBG000BP52R2,BBG000BP52R2,BBG001S5T7X2,US,Equity,MMM,Common Stock,Common Stock,NaN
4,88 ENERGY LTD,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,...,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19731,ZYNERBA PHARMACEUTICALS INC,ZYNE,USA,USD,US,US98986X1090,Zynerba Pharmaceuticals Inc,Common Stock,1.170,2022-08-03,...,ZYNE,BBG007BBS8B7,BBG007BBS8B7,BBG007BBS8C6,US,Equity,ZYNE,Common Stock,Common Stock,NaN
19732,ZYNEX INC,ZYXI,USA,USD,US,US98986M1036,Zynex Inc,Common Stock,8.400,2022-08-03,...,ZYXI,BBG000BJBXZ2,BBG000BJBXZ2,BBG001S7T7V0,US,Equity,ZYXI,Common Stock,Common Stock,NaN
19733,ZYNGA INC - CL A,NaN,NaN,NaN,NaN,US98986T1088,NaN,NaN,NaN,NaN,...,ZNGAEUR,BBG005YZL376,BBG005YZL2R6,BBG001T24RN2,XS,Equity,ZNGAEUR,Common Stock,Common Stock,NaN
19734,ZYTRONIC PLC,ZYT,UK,GBX,LSE,GB0006971013,Zytronic,Common Stock,140.000,2022-08-03,...,ZYT,BBG000BZPV76,BBG000BZPTJ8,BBG001SFLP13,LN,Equity,ZYT,Common Stock,Common Stock,NaN


## EOD vs 19.7k

In [10]:

df_Name = df[['ISIN', 'Name_197', 'Name-EOD']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'Name-EOD']].dropna(subset=['Name-EOD'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['Name-EOD'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])

df_matchFinal = df_match[df_match['fuzz_socre1']<60]
df_matchFinal.to_csv('./Data/eod-197.csv', index=False)
df_matchFinal

13016
progress: 0.00%
progress: 10.00%
progress: 19.99%
progress: 29.99%
progress: 39.98%
progress: 49.98%
progress: 59.97%
progress: 69.97%
progress: 79.96%
progress: 89.96%
progress: 99.95%


,ISIN,Name_197,Name-EOD,fuzz_socre1
38,JP3108120001,ACRODEA INC,Why How Do Co Inc,50
53,LU1250154413,ADO PROPERTIES SA,Adler Group SA,58
78,US0077371096,AEROCENTURY CORP,Mega Matrix Corp,25
165,US0185811082,ALLIANCE DATA SYSTEMS CORP,Bread Financial Holdings Inc,26
223,US0259321042,AMERICAN FINANCIAL GROUP INC,Aquatic Foods Group plc,51
...,...,...,...,...
12934,US98585L1008,YIREN DIGITAL LTD - SPS ADR,Yirendai Ltd,49
12942,CNE100002W92,YOUON TECHNOLOGY CO LTD-A,Changzhou Youon Pubilic Bicycle System Co Ltd,49
12970,CH0276534614,ZEHNDER GROUP AG-RG,Zehnder,54
12996,CNE000000FL2,ZHONGTIAN FINANCIAL GROUP -A,Zhongtian Urban Development Group Co Ltd,54


## Investing.com vs 19.7k

In [11]:
df_Name = df[['ISIN', 'Name_197', 'full_name-Inv']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'full_name-Inv']].dropna(subset=['full_name-Inv'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['full_name-Inv'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])

df_matchFinal = df_match[df_match['fuzz_socre1']<60]
df_matchFinal.to_csv('./Data/inv-197.csv', index=False)
df_matchFinal


6141
progress: 0.00%
progress: 10.00%
progress: 20.00%
progress: 30.00%
progress: 39.99%
progress: 49.99%
progress: 59.99%
progress: 69.99%
progress: 79.99%
progress: 219.96%


,ISIN,Name_197,full_name-Inv,fuzz_socre1
246,IL0010806136,ANALYST I.M.S.,Analyst IMS Investment Management Services Ltd,44
562,RU000A0HL5M1,BELUGA,Beluga Group PAO,55
2779,US5006432000,KORN FERRY,Korn/Ferry International,59
2965,IL0004730177,LUZON GROUP,Amos Luzon Development and Energy Group Ltd,41
3794,PLPKO0000016,PKO BANK POLSKI SA,Powszechna Kasa Oszczednosci Bank Polski SA,56
4167,FR0006239109,SCBSM,Soc Centrale Bois Scieries Manche SA,15


## IB vs 19.7k

In [12]:
df_Name = df[['ISIN', 'Name_197', 'longName-IB']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'longName-IB']].dropna(subset=['longName-IB'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['longName-IB'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])

df_matchFinal = df_match[df_match['fuzz_socre1']<60]
df_matchFinal.to_csv('./Data/ib-197.csv', index=False)
df_matchFinal

16912
progress: 0.00%
progress: 10.00%
progress: 20.00%
progress: 30.00%
progress: 40.00%
progress: 49.99%
progress: 59.99%
progress: 69.99%
progress: 89.99%
progress: 99.99%


,ISIN,Name_197,longName-IB,fuzz_socre1


## FIGI vs 19.7k

In [13]:
df_Name = df[['ISIN', 'Name_197', 'longName-FIGI']].dropna(subset=['Name_197'])
df_Name = df_Name[['ISIN', 'Name_197', 'longName-FIGI']].dropna(subset=['longName-FIGI'])

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name_197'],row['longName-FIGI'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_matchFinal = df_match[df_match['fuzz_socre1']<60]
df_matchFinal.to_csv('./Data/figi-197.csv', index=False)
df_matchFinal

15485
progress: 0.00%
progress: 10.00%
progress: 19.99%
progress: 29.99%
progress: 39.99%
progress: 49.98%
progress: 59.98%
progress: 69.98%
progress: 79.97%
progress: 99.97%


,ISIN,Name_197,longName-FIGI,fuzz_socre1
2,JP3160590000,A DOT CO LTD,BIRDMAN INC,26
38,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,32
53,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,58
78,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,25
121,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,57
...,...,...,...,...
16577,US74838C1062,QUIKSILVER INC,BOARDRIDERS INC,21
16581,US7472623013,QVC INC 6 3/8 09/13/67 PFD,QVC INC,42
16690,SE0010520155,ROLLING OPTICS HOLDING AB,ICTA AB,31
17055,US7847882005,"SYNTHETIC FIXED-INCOME SEC STRATS 2004-2, 6.50...",STRATS-FON-04-2,36


## Merge after manually check

In [12]:
path = "./Data/withFIGI.csv"

df_figiCk = pd.read_csv("./Data/figi-197.csv")
df_ibCk = pd.read_csv("./Data/ib-197.csv") 
df_eodCk = pd.read_csv("./Data/eod-197.csv") 
df_invCk = pd.read_csv("./Data/inv-197.csv") 

In [13]:
df_ck = pd.merge(df_figiCk, df_ibCk, how='outer', on=['ISIN', 'Name_197'])
df_ck = pd.merge(df_ck, df_eodCk, how='outer', on=['ISIN', 'Name_197'])
df_ck = pd.merge(df_ck, df_invCk, how='outer', on=['ISIN', 'Name_197'])

df_ck = df_ck[['ISIN', 'Name_197', 'longName-FIGI', 'check-FIGI', 'longName-IB', 'check-IB', 'Name-EOD', 'check-EOD', 'full_name-Inv', 'Check-Inv']]

df_ck

,ISIN,Name_197,longName-FIGI,check-FIGI,longName-IB,check-IB,Name-EOD,check-EOD,full_name-Inv,Check-Inv
0,JP3160590000,A DOT CO LTD,BIRDMAN INC,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,0.0,NaN,NaN,Why How Do Co Inc,0.0,NaN,NaN
2,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,0.0,NaN,NaN,Adler Group SA,0.0,NaN,NaN
3,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,0.0,NaN,NaN,Mega Matrix Corp,0.0,NaN,NaN
4,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
405,CNE100002W92,YOUON TECHNOLOGY CO LTD-A,NaN,NaN,NaN,NaN,Changzhou Youon Pubilic Bicycle System Co Ltd,1.0,NaN,NaN
406,CH0276534614,ZEHNDER GROUP AG-RG,NaN,NaN,NaN,NaN,Zehnder,1.0,NaN,NaN
407,CNE000000FL2,ZHONGTIAN FINANCIAL GROUP -A,NaN,NaN,NaN,NaN,Zhongtian Urban Development Group Co Ltd,1.0,NaN,NaN
408,HU0000074844,ZWACK UNICUM RT,NaN,NaN,NaN,NaN,Zwack Unicum Likoripari es Kereskedelmi Nyrt.,1.0,NaN,NaN


In [14]:
df_ck['Not_Match'] = ""

df_ck.loc[df_ck['check-FIGI']==0,'Not_Match'] = 'FIGI;'
df_ck.loc[df_ck['check-IB']==0,'Not_Match'] = 'IB;' + df_ck['Not_Match'].astype(str)
df_ck.loc[df_ck['check-EOD']==0,'Not_Match'] = 'EOD;' + df_ck['Not_Match'].astype(str)
df_ck.loc[df_ck['Check-Inv']==0,'Not_Match'] = 'Inv;' + df_ck['Not_Match'].astype(str)

df_ck['Not_Match'] = df_ck['Not_Match'].replace('', np.nan)

df_ck

,ISIN,Name_197,longName-FIGI,check-FIGI,longName-IB,check-IB,Name-EOD,check-EOD,full_name-Inv,Check-Inv,Not_Match
0,JP3160590000,A DOT CO LTD,BIRDMAN INC,0.0,NaN,NaN,NaN,NaN,NaN,NaN,FIGI;
1,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,0.0,NaN,NaN,Why How Do Co Inc,0.0,NaN,NaN,EOD;FIGI;
2,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,0.0,NaN,NaN,Adler Group SA,0.0,NaN,NaN,EOD;FIGI;
3,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,0.0,NaN,NaN,Mega Matrix Corp,0.0,NaN,NaN,EOD;FIGI;
4,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
405,CNE100002W92,YOUON TECHNOLOGY CO LTD-A,NaN,NaN,NaN,NaN,Changzhou Youon Pubilic Bicycle System Co Ltd,1.0,NaN,NaN,NaN
406,CH0276534614,ZEHNDER GROUP AG-RG,NaN,NaN,NaN,NaN,Zehnder,1.0,NaN,NaN,NaN
407,CNE000000FL2,ZHONGTIAN FINANCIAL GROUP -A,NaN,NaN,NaN,NaN,Zhongtian Urban Development Group Co Ltd,1.0,NaN,NaN,NaN
408,HU0000074844,ZWACK UNICUM RT,NaN,NaN,NaN,NaN,Zwack Unicum Likoripari es Kereskedelmi Nyrt.,1.0,NaN,NaN,NaN


In [15]:
df_matchFinal = df_ck[df_ck['Not_Match']!='']
df_matchFinal

,ISIN,Name_197,longName-FIGI,check-FIGI,longName-IB,check-IB,Name-EOD,check-EOD,full_name-Inv,Check-Inv,Not_Match
0,JP3160590000,A DOT CO LTD,BIRDMAN INC,0.0,NaN,NaN,NaN,NaN,NaN,NaN,FIGI;
1,JP3108120001,ACRODEA INC,THE WHY HOW DO COMPANY INC,0.0,NaN,NaN,Why How Do Co Inc,0.0,NaN,NaN,EOD;FIGI;
2,LU1250154413,ADO PROPERTIES SA,ADLER GROUP SA,0.0,NaN,NaN,Adler Group SA,0.0,NaN,NaN,EOD;FIGI;
3,US0077371096,AEROCENTURY CORP,MEGA MATRIX CORP,0.0,NaN,NaN,Mega Matrix Corp,0.0,NaN,NaN,EOD;FIGI;
4,JP3102000001,AISIN SEIKI CO LTD,AISIN CORP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
405,CNE100002W92,YOUON TECHNOLOGY CO LTD-A,NaN,NaN,NaN,NaN,Changzhou Youon Pubilic Bicycle System Co Ltd,1.0,NaN,NaN,NaN
406,CH0276534614,ZEHNDER GROUP AG-RG,NaN,NaN,NaN,NaN,Zehnder,1.0,NaN,NaN,NaN
407,CNE000000FL2,ZHONGTIAN FINANCIAL GROUP -A,NaN,NaN,NaN,NaN,Zhongtian Urban Development Group Co Ltd,1.0,NaN,NaN,NaN
408,HU0000074844,ZWACK UNICUM RT,NaN,NaN,NaN,NaN,Zwack Unicum Likoripari es Kereskedelmi Nyrt.,1.0,NaN,NaN,NaN


In [18]:
df_matchFinal[['ISIN','Not_Match']]

,ISIN,Not_Match
0,JP3160590000,FIGI;
1,JP3108120001,EOD;FIGI;
2,LU1250154413,EOD;FIGI;
3,US0077371096,EOD;FIGI;
4,JP3102000001,NaN
...,...,...
405,CNE100002W92,NaN
406,CH0276534614,NaN
407,CNE000000FL2,NaN
408,HU0000074844,NaN


In [24]:
df = df.drop('Not_Match', axis=1)
df

,Name_197,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,...,longName-FIGI,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI
0,1ST GROUP LTD,1ST,Australia,AUD,AU,AU0000001ST0,1stAvailable Ltd,Common Stock,0.010,2022-08-03,...,1ST GROUP LTD,1ST,BBG008HNLRF4,BBG008HNLRF4,BBG008HNLRG3,AU,Equity,1ST,Common Stock,Common Stock
1,2U INC,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,...,2U INC,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock
2,3DX INDUSTRIES INC,DDDX,USA,USD,US,US88556A1007,3Dx Industries Inc,Common Stock,0.060,2022-08-02,...,3DX INDUSTRIES INC,DDDX,BBG000C129N6,BBG000C129N6,BBG001STRH79,US,Equity,DDDX,Common Stock,Common Stock
3,3M CO,MMM,USA,USD,US,US88579Y1010,3M Company,Common Stock,141.750,2022-08-02,...,3M CO,MMM,BBG000BP52R2,BBG000BP52R2,BBG001S5T7X2,US,Equity,MMM,Common Stock,Common Stock
4,88 ENERGY LTD,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,...,88 ENERGY LTD,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19731,ZYNERBA PHARMACEUTICALS INC,ZYNE,USA,USD,US,US98986X1090,Zynerba Pharmaceuticals Inc,Common Stock,1.170,2022-08-03,...,ZYNERBA PHARMACEUTICALS INC,ZYNE,BBG007BBS8B7,BBG007BBS8B7,BBG007BBS8C6,US,Equity,ZYNE,Common Stock,Common Stock
19732,ZYNEX INC,ZYXI,USA,USD,US,US98986M1036,Zynex Inc,Common Stock,8.400,2022-08-03,...,ZYNEX INC,ZYXI,BBG000BJBXZ2,BBG000BJBXZ2,BBG001S7T7V0,US,Equity,ZYXI,Common Stock,Common Stock
19733,ZYNGA INC - CL A,NaN,NaN,NaN,NaN,US98986T1088,NaN,NaN,NaN,NaN,...,ZYNGA INC - CL A,ZNGAEUR,BBG005YZL376,BBG005YZL2R6,BBG001T24RN2,XS,Equity,ZNGAEUR,Common Stock,Common Stock
19734,ZYTRONIC PLC,ZYT,UK,GBX,LSE,GB0006971013,Zytronic,Common Stock,140.000,2022-08-03,...,ZYTRONIC PLC,ZYT,BBG000BZPV76,BBG000BZPTJ8,BBG001SFLP13,LN,Equity,ZYT,Common Stock,Common Stock


In [25]:
df = pd.merge(df, df_matchFinal[['ISIN','Not_Match']], how='outer', on='ISIN')

df

,Name_197,Code-EOD,Country-EOD,Currency-EOD,Exchange-EOD,ISIN,Name-EOD,Type-EOD,previousClose-EOD,previousCloseDate-EOD,...,symbol-FIGI,figi,compositeFIGI,shareClassFIGI,exchCode-FIGI,marketSector-FIGI,securityDescription-FIGI,securityType-FIGI,securityType2-FIGI,Not_Match
0,1ST GROUP LTD,1ST,Australia,AUD,AU,AU0000001ST0,1stAvailable Ltd,Common Stock,0.010,2022-08-03,...,1ST,BBG008HNLRF4,BBG008HNLRF4,BBG008HNLRG3,AU,Equity,1ST,Common Stock,Common Stock,NaN
1,2U INC,TWOU,USA,USD,US,US90214J1016,2U Inc,Common Stock,9.200,2022-08-02,...,TWOU,BBG001KS9450,BBG001KS9450,BBG001V0RR30,US,Equity,TWOU,Common Stock,Common Stock,NaN
2,3DX INDUSTRIES INC,DDDX,USA,USD,US,US88556A1007,3Dx Industries Inc,Common Stock,0.060,2022-08-02,...,DDDX,BBG000C129N6,BBG000C129N6,BBG001STRH79,US,Equity,DDDX,Common Stock,Common Stock,NaN
3,3M CO,MMM,USA,USD,US,US88579Y1010,3M Company,Common Stock,141.750,2022-08-02,...,MMM,BBG000BP52R2,BBG000BP52R2,BBG001S5T7X2,US,Equity,MMM,Common Stock,Common Stock,NaN
4,88 ENERGY LTD,88E,UK,GBX,LSE,AU00000088E2,88 Energy Ltd,Common Stock,0.625,2022-08-03,...,POQ,BBG000BWTTK9,BBG000BWTTK9,BBG001SCV1F7,GR,Equity,POQ,Common Stock,Common Stock,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19563,ZYTRONIC PLC,ZYT,UK,GBX,LSE,GB0006971013,Zytronic,Common Stock,140.000,2022-08-03,...,ZYT,BBG000BZPV76,BBG000BZPTJ8,BBG001SFLP13,LN,Equity,ZYT,Common Stock,Common Stock,NaN
19564,ZZ CAPITAL INTERNATIONAL LTD,NaN,NaN,NaN,NaN,KYG9895T1085,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19565,NaN,NaN,NaN,NaN,NaN,RU000A0HL5M1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19566,NaN,NaN,NaN,NaN,NaN,RU000A0JPV70,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.to_csv('./Data/withFIGI.csv')

In [27]:
df_notMatch = df[df['Not_Match']=='EOD;FIGI;']
df_notMatch = df_notMatch[['ISIN', 'Name-EOD', 'longName-FIGI']]

df_notMatch

,ISIN,Name-EOD,longName-FIGI
2258,JP3108120001,Why How Do Co Inc,THE WHY HOW DO COMPANY INC
2323,LU1250154413,Adler Group SA,ADLER GROUP SA
2381,US0077371096,Mega Matrix Corp,MEGA MATRIX CORP
2656,US0185811082,Bread Financial Holdings Inc,BREAD FINANCIAL HOLDINGS INC
3762,JP3160770008,Wow World Inc,WOW WORLD INC
3783,IL0011016669,Abra Information Technologies Ltd,ABRA INFORMATION TECHNOLOGIE
4099,CNE0000005N4,New Guomai Digital Culture Co Ltd,NEW GUOMAI DIGITAL CULTURE-A
4203,US09074B1070,Renovare Environmental Inc,RENOVARE ENVIRONMENTAL INC
4620,US1143401024,Azenta Inc,AZENTA INC
4720,US1270971039,Coterra Energy Inc,COTERRA ENERGY INC


In [55]:
df_Name = df_notMatch.copy()

print(len(df_Name))

allD = []

for index, row in df_Name.iterrows():
    # calculate nlp scores
    fuzz_socre1 = fuzz.token_sort_ratio(row['Name-EOD'],row['longName-FIGI'])
            
    row['fuzz_socre1'] = fuzz_socre1
    allD.append(row)
       

    if index % (len(df_Name)//10) == 0:
        print("progress: %.2f" % (100*index/len(df_Name)) + "%")

df_match = pd.DataFrame(allD).drop_duplicates(subset=['ISIN'])
df_match[df_match['fuzz_socre1']<60]

50
progress: 330.00%
progress: 1190.00%
progress: 5020.00%
progress: 5120.00%
progress: 6820.00%
progress: 8490.00%
progress: 13660.00%
progress: 14140.00%
progress: 25470.00%


,ISIN,Name-EOD,longName-FIGI,fuzz_socre1
2051,GB00BYRH4982,Rockwood Realisation PLC,ROCKWOOD STRATEGIC PLC,52
6830,US1255258576,Creative Media & Community Trust Corporation P...,CREATIVE MEDIA & CMNTY,49
7070,US74913G8731,Qwest Corp 6 75 Notes Expiry 2057,QWEST CORP,47
12735,US94770G1058,Veridetx Corp,ONEMETA AI,35
